In [1]:
from scipy.stats import gamma
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.special import comb
import os
import time

In [2]:
plt.rcParams['figure.figsize'] = [10,10]
plt.rcParams['figure.dpi'] = 100

In [3]:
filename = '[condition 2] The protein-peptide table for Result from [neo4j] Start neo4j database'

# Read

In [4]:
start = time.time()

if filename not in os.listdir('./output2'):
    os.mkdir('./output2/%s'%filename)
f = open('./input2/%s.tsv'%filename,'r')

f.readline()#不要的

f_write = open('./output2/%s/2.LPGF.csv'%filename,'w')
f_write.write('p.accession,p.type,target.LPGF,decoy.LPGF\n')

#Create list
p_accession_ls = []
p_type_ls = []
target_LPGF_ls = []
decoy_LPGF_ls = []

#第一個
f1 = f.readline()
f1 = f1.strip().split('\t')
protein = f1[0]
ptype = f1[1]
if (f1[3] == 'target')&(f1[5] == 'true'):
    score_T_True = [float(f1[4])]
    score_T_False = []
    score_D_True = []
    score_D_False = []       
elif (f1[3] == 'decoy')&(f1[5] == 'true'):
    score_T_True = []
    score_T_False = []    
    score_D_True = [float(f1[4])]
    score_D_False = []    
elif (f1[3] == 'target')&(f1[5] == 'false'):
    score_T_True = []
    score_T_False = [float(f1[4])]
    score_D_True = []
    score_D_False = []   
elif (f1[3] == 'decoy')&(f1[5] == 'false'):
    score_T_True = []
    score_T_False = []     
    score_D_True = []
    score_D_False = [float(f1[4])]    

#第二個以後
while True:
    f1 = f.readline()
    if f1 == '':
        #算最後
        #計算Target
        LPF = sum(score_T_True)
        m = len(score_T_True)
        n = m + len(score_T_False)
        if m > 0:
            LPGF_target = -np.log10((1-gamma.cdf(LPF*np.log(10), a=m, scale=1))*comb(n,m))
        else:
            if n != 0:
                LPM = max(score_T_True+score_T_False)
                LPGF_target = -np.log10(1-(1-10**(-LPM))**n)
            else:
                LPGF_target = 'NA'
        #計算Decoy
        LPF = sum(score_D_True)
        m = len(score_D_True)
        n = m + len(score_D_False)
        if m > 0:
            LPGF_decoy = -np.log10((1-gamma.cdf(LPF*np.log(10), a=m, scale=1))*comb(n,m))
        else:
            if n != 0:
                LPM = max(score_D_True+score_D_False)
                LPGF_decoy = -np.log10(1-(1-10**(-LPM))**n)
            else:
                LPGF_decoy = 'NA'
        f_write.write('%s,%s,%s,%s\n'%(protein,ptype,LPGF_target,LPGF_decoy))  
        p_accession_ls.append(protein)
        p_type_ls.append(ptype)
        target_LPGF_ls.append(LPGF_target)
        decoy_LPGF_ls.append(LPGF_decoy)
        #關閉
        f.close()
        break
        
        
        
    f1 = f1.strip().split('\t')

    
    #如果蛋白一樣就儲存
    if f1[0] == protein:
        if (f1[3] == 'target')&(f1[5] == 'true'):
            score_T_True.append(float(f1[4]))
        elif (f1[3] == 'decoy')&(f1[5] == 'true'):
            score_D_True.append(float(f1[4]))
        elif (f1[3] == 'target')&(f1[5] == 'false'):
            score_T_False.append(float(f1[4]))
        elif (f1[3] == 'decoy')&(f1[5] == 'false'):
            score_D_False.append(float(f1[4]))    
    #如果蛋白不一樣就計算 + 寫 + 儲存新的
    else:
        #計算Target
        LPF = sum(score_T_True)
        m = len(score_T_True)
        n = m + len(score_T_False)
        if m > 0:
            LPGF_target = -np.log10((1-gamma.cdf(LPF*np.log(10), a=m, scale=1))*comb(n,m))
        else:
            if n != 0:
                LPM = max(score_T_True+score_T_False)
                LPGF_target = -np.log10(1-(1-10**(-LPM))**n)
            else:
                LPGF_target = 'NA'
        #計算Decoy
        LPF = sum(score_D_True)
        m = len(score_D_True)
        n = m + len(score_D_False)
        if m > 0:
            LPGF_decoy = -np.log10((1-gamma.cdf(LPF*np.log(10), a=m, scale=1))*comb(n,m))
        else:
            if n != 0:
                LPM = max(score_D_True+score_D_False)
                LPGF_decoy = -np.log10(1-(1-10**(-LPM))**n)
            else:
                LPGF_decoy = 'NA'
        f_write.write('%s,%s,%s,%s\n'%(protein,ptype,LPGF_target,LPGF_decoy))     
        p_accession_ls.append(protein)
        p_type_ls.append(ptype)
        target_LPGF_ls.append(LPGF_target)
        decoy_LPGF_ls.append(LPGF_decoy)        
        
        #儲存新的
        protein = f1[0]
        ptype = f1[1]
        if (f1[3] == 'target')&(f1[5] == 'true'):
            score_T_True = [float(f1[4])]
            score_T_False = []
            score_D_True = []
            score_D_False = []       
        elif (f1[3] == 'decoy')&(f1[5] == 'true'):
            score_T_True = []
            score_T_False = []    
            score_D_True = [float(f1[4])]
            score_D_False = []    
        elif (f1[3] == 'target')&(f1[5] == 'false'):
            score_T_True = []
            score_T_False = [float(f1[4])]
            score_D_True = []
            score_D_False = []   
        elif (f1[3] == 'decoy')&(f1[5] == 'false'):
            score_T_True = []
            score_T_False = []     
            score_D_True = []
            score_D_False = [float(f1[4])]    
f_write.close()


end = time.time()
print('used %.2f sec.'%(end-start))

C:\Users\chienhua\anaconda3\lib\site-packages\ipykernel_launcher.py:104: RuntimeWarning: divide by zero encountered in log10
C:\Users\chienhua\anaconda3\lib\site-packages\ipykernel_launcher.py:104: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\chienhua\anaconda3\lib\site-packages\ipykernel_launcher.py:116: RuntimeWarning: divide by zero encountered in log10


used 8.19 sec.


In [21]:
min([i for i in target_LPGF_ls if (str(i)!='inf')&(i!='NA')])

-3.1038148338520326

In [23]:
Target_score = target_LPGF_ls.copy()
Decoy_score = decoy_LPGF_ls.copy()
#轉換Target
Target_score_rmInf_rmNA = []
max_score = max([i for i in target_LPGF_ls if (str(i)!='inf')&(i!='NA')])
min_score = min([i for i in target_LPGF_ls if (str(i)!='inf')&(i!='NA')])
for i in Target_score:
    if str(i) == 'inf':
        Target_score_rmInf_rmNA.append(max_score+1)
    elif str(i) == 'NA':
        Target_score_rmInf_rmNA.append(min_score-1)
    else:
        Target_score_rmInf_rmNA.append(i)   
        
#轉換Decoy
Decoy_score_rmInf_rmNA = []
max_score = max([i for i in decoy_LPGF_ls if (str(i)!='inf')&(i!='NA')])
min_score = min([i for i in decoy_LPGF_ls if (str(i)!='inf')&(i!='NA')])
for i in Decoy_score:
    if str(i) == 'inf':
        Decoy_score_rmInf_rmNA.append(max_score+1)
    elif str(i) == 'NA':
        Decoy_score_rmInf_rmNA.append(min_score-1)
    else:
        Decoy_score_rmInf_rmNA.append(i)  

# CHECK
### 檢查peptide序列是否有重複assign到不同的蛋白質

In [5]:
f1 = pd.read_csv('./input2/%s.tsv'%filename,sep='\t')

In [8]:
f2 = f1[f1.duplicated(['pep.modpepseq'],keep=False)]
f2_target = f2[f2['cs.type']=='target']
f2_target = f2_target[f2_target .duplicated(['pep.modpepseq'],keep=False)]
del f1, f2

In [9]:
PEP = np.unique(f2_target['pep.modpepseq'])

f3 = open('./output2/%s/(CHECK)one-peptide-to-mutiple-protein.csv'%filename,'w')
f3.write('pep.modpepseq,p.accession,cs.type\n')
err_count = 0

if len(PEP)!= 0:
    for pep in PEP:
        if sorted(list(f2_target[f2_target['pep.modpepseq']==pep]['cs.type'])) == ['decoy','target']:
            #print(pep,'OK')
            continue
        elif list(set(f2_target[f2_target['pep.modpepseq']==pep]['cs.type'])) == ['decoy']:
            continue
        else:
            err_count +=1
            proteins = ';'.join(list(f2_target[f2_target['pep.modpepseq']==pep]['p.accession']))
            cstype = ';'.join(list(f2_target[f2_target['pep.modpepseq']==pep]['cs.type']))
            f3.write('%s,%s,%s\n'%(pep,proteins,cstype))
            
f3.close()
print('Error Number of Peptide Counts=%s'%err_count)

Error Number of Peptide Counts=0
